# dcm/nii, mat/py 공간표현 차이


# 기본 축(axis)

- dcm/nii : x y z   (= dicominfo : PatientPosicions, PatientOrientations, ImageSize..)  

- matlab :  y x z   (= dicomread : image, dose.. 실제 3D 데이터)  
- python :  z y x   (= dicomread : image, dose.. 실제 3D 데이터)  


# 공간 좌표계

> CT 좌표계  
>   
> x : right-left          (R-L)  
> y : anterior-posterior  (A-P)  
> z : inferior-superior   (I-S)  

- DICOM (LPS)  

    x : R->L  
    y : A->P  
    z : I->S  

- NIfTI (RAS)  

    x : L->R  
    y : P->A  
    z : I->S  

### + 행 순서 차이  

- NifTi (I열 J행 K슬라이스)  

    i(=x) : 왼->오  
    j(=y) : 아래->위  

- DICOM  

    i(=x) : 왼->오  
    j(=y) : 위->아래  


# python(LPS) -> nii(RAS) 변환

## affine transformation matrix  
: 복셀 좌표(r,c,s) pixel -> 물리적 좌표(x,y,z) mm 변환

$
A =
\begin{bmatrix}
X_x \cdot \Delta x & Y_x \cdot \Delta y & Z_x \cdot \Delta z & X_0 \\
X_y \cdot \Delta x & Y_y \cdot \Delta y & Z_y \cdot \Delta z & Y_0 \\
X_z \cdot \Delta x & Y_z \cdot \Delta y & Z_z \cdot \Delta z & Z_0 \\
0 & 0 & 0 & 1
\end{bmatrix}
$  

$
\begin{array}{rl}
X_0             & : X \text{ offset} \\
[X_x, X_y, X_z] & : X \text{ orientation} \\
\Delta x        & : X \text{ spacing} \\
\end{array}
$


## dcm2niix 결과는 RAS가 아니다 (LAS)

LAS (LPS + L-R반전)
(kaggle dataset, opensource(dcm2niix) 사용결과)  


+ dcm2niix 방식 : LPS 유지 + (L-R 반전), LPS를 유지하려고 하는데 ITK 같은 뷰어에서 반대로 읽히지 않으려고  L-R 반전 적용


# VTK, ITK
VTK : Visualization Toolkit  
ITK : Insight Toolkit  

- DICOM : 이미지 좌상단 원점  
- NifTi, VTK,ITK : 이미지 좌하단 원점  

In [1]:
''' import '''
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

import pydicom
import nibabel as nib

from dicom import dcmread_volume

In [2]:
root_path = os.path.join(os.getcwd(), '..', '..',  'dicom_matlab', 'data', 'patient-example')

ct_folder = None
rtdose_folder = None
rtst_folder = None

ct_files = []
rtdose_files = []
rtst_files = []

# folders
for folder in os.listdir(root_path):
    if 'CT' in folder:
        ct_folder = os.path.join(root_path, folder)
    if 'RTDOSE' in folder:
        rtdose_folder = os.path.join(root_path, folder)
    if 'RTst' in folder:
        rtst_folder = os.path.join(root_path, folder)

# files
for file in os.listdir(ct_folder): # CT
    if file.endswith('.dcm'):
        ct_files.append(os.path.join(ct_folder, file))

for file in os.listdir(rtdose_folder): # RT dose
    if file.endswith('.dcm'):
        rtdose_files.append(os.path.join(rtdose_folder, file))

for file in os.listdir(rtdose_folder): # RT structure
    if file.endswith('.dcm'):
        rtst_files.append(os.path.join(rtst_folder, file))

# file
rtdose_file = rtdose_files[0]
rtst_file = rtst_files[0]

# CT

In [67]:
img_origin = np.zeros(3, dtype=np.double)
img_spacing = np.zeros(3, dtype=np.double)
img_size = np.zeros(3, dtype=np.int16)


''' CT meta '''
ct_info_tmp = pydicom.dcmread(ct_files[0])

rescale_slope = ct_info_tmp.RescaleSlope
rescale_intercept = ct_info_tmp.RescaleIntercept

slice_thickness = ct_info_tmp.SliceThickness


''' CT img '''
image_raw, spatial = dcmread_volume(ct_folder)

img_origin[:] = spatial.PatientPositions[0, :]
img_spacing[:2] = spatial.PixelSpacings[0, :]
img_spacing[2] = slice_thickness
img_size[:] = spatial.ImageSize         # matlab : 512(y=r) 512(x=c) 337

image = (image_raw*rescale_slope) + rescale_intercept
# image = np.transpose(image, (1,0,2)) # y x z -> x y z

In [ ]:
''' anterior-posterior (y, 앞->뒤)'''
for i in range(img_size[0]):
    clear_output(wait=True)
    plt.imshow(image[i,:,:])
    plt.show()

In [ ]:
''' right-left (x, 오->왼) ''' # segittal에서 R->L, L->R은 구분할 수 없다
for i in range(img_size[1]):
    clear_output(wait=True)
    plt.imshow(image[:,i,:])
    plt.show()

In [ ]:
''' inferior-superior (z, 아래->위) '''
for i in range(img_size[2]):
    clear_output(wait=True)
    plt.imshow(image[:,:,i])
    plt.show()

# RT dose

In [ ]:
# rtdose_origin = np.zeros(3, dtype=np.double)
# rtdose_spacing = np.zeros(3, dtype=np.double)
# rtdose_size = np.zeros(3, dtype=np.int16)


# ''' RT dose meta'''
# rtdose_info = pydicom.dcmread(rtdose_file)

# grid_scale = rtdose_info.DoseGridScaling
# slice_thickness = rtdose_info.SliceThickness


# ''' RT dose img '''
# rtdose_raw = rtdose_info.pixel_array
# # rtdose_raw = np.transpose(rtdose_raw, (1,2,0)) # z y x -> y x z

# rtdose_origin[:] = rtdose_info.ImagePositionPatient
# rtdose_spacing[:2] = rtdose_info.PixelSpacing
# rtdose_spacing[2] = slice_thickness
# rtdose_size[:] = rtdose_raw.shape   # python(z y x) : 317(z) 143(y=r) 267(x=c)   / matlab(y x z) : 143(y=h,r) 267(x=w,c) 317 ----> dcm/nii(x y z)

# rtdose = rtdose_raw*grid_scale

In [ ]:
# ''' inferior-superior (z, 아래->위) '''
# for i in range(rtdose_size[0]):
#     clear_output(wait=True)
#     plt.imshow(rtdose[i,:,:])
#     plt.show()

In [ ]:
# ''' anterior-posterior (y, 앞->뒤) '''
# for i in range(rtdose_size[1]):
#     clear_output(wait=True)
#     plt.imshow(rtdose[:,i,:])
#     plt.show()

In [ ]:
# ''' right-left (x, 왼->오) '''
# for i in range(rtdose_size[2]):
#     clear_output(wait=True)
#     plt.imshow(rtdose[:,:,i])
#     plt.show()